In [ ]:
!pip install geopandas
!pip install contextily

In [ ]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [1]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [2]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [5]:
iso3 = "HND"  ##ARG URY
file_path = f"/projects/my-public-bucket/AOIs/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [6]:
json_files = tileindex_df['s3path']
AOIs = json_files
print(len(AOIs))
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

22


'tile_num_16275'

In [29]:
AOIs[1]

'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16276.geojson'

In [14]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2A subsetter for tile_num_15735!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15736!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15913!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15914!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15915!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15916!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15917!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16093!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16094!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16095!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16096!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16097!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16273!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16274!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16275!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16276!!
Thanks, I'll run the GED

In [11]:
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
## if necessary ## re-run failed subsets
#BRA
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22982.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L2A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
    #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
    #columns="all",
    query="quality_flag == 1",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'b0ffcdc2-6e99-4a7b-8587-e76c1ca7610f'

In [8]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


934


In [9]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/09/08"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_22080_L2A.gpkg...
Copying tile_num_22259_L2A.gpkg...
Copying tile_num_23160_L2A.gpkg...
Copying tile_num_22620_L2A.gpkg...
